### Code of functions to create orders and get balance

In [14]:
import pandas as pd
from tinkoff.invest import Client, InstrumentStatus, SharesResponse, InstrumentIdType
from tinkoff.invest.services import InstrumentsService, MarketDataService
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List
import random

import os
import datetime
from tinkoff.invest import CandleInterval

from tinkoff.invest import Client
import seaborn as sns
sns.set(palette='summer')

In [15]:
token = 'YOUR_TOKEN'

In [16]:
df = pd.read_csv('./companies_fixed.csv')
df['cap'] = df['cap'].astype(float)

In [17]:
from tinkoff.invest import Client, RequestError, PortfolioResponse, PositionsResponse, PortfolioPosition


import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


"""
Для видео по get_portfolio
https://tinkoff.github.io/investAPI/operations/#portfoliorequest
https://tinkoff.github.io/investAPI
https://github.com/Tinkoff/invest-python
"""
def run():
    try:
        with Client(token) as client:
            acc_id = '2210190886'
            u = client.market_data.get_last_prices(figi=['BBG004730N88'])
            usdrur = cast_money(u.last_prices[0].price)
            r : PortfolioResponse = client.operations.get_portfolio(account_id=acc_id)
            op = client.operations.get_portfolio(account_id=acc_id)
            return r, op

    except RequestError as e:
        print(str(e))



def cast_money(v):
    return v.units + v.nano / 1e9 # nano - 9 нулей

In [18]:
def portfolio_pose_todict(p : PortfolioPosition):
    r = {
        'figi': p.figi,
        'quantity': cast_money(p.quantity),
        'expected_yield': cast_money(p.expected_yield),
        'instrument_type': p.instrument_type,
        'average_buy_price': cast_money(p.average_position_price),
        'currency': p.average_position_price.currency,
        'nkd': cast_money(p.current_nkd),
    }
 
    r['sell_sum'] = (r['average_buy_price']*r['quantity']) + r['expected_yield'] + (r['nkd']*r['quantity'])
    r['comission'] = r['sell_sum']*0.003
    r['tax'] = r['expected_yield']*0.013 if r['expected_yield'] > 0 else 0
 
    return r

In [19]:
r, op = run()
r

PortfolioResponse(total_amount_shares=MoneyValue(currency='rub', units=64776, nano=820000000), total_amount_bonds=MoneyValue(currency='rub', units=0, nano=0), total_amount_etf=MoneyValue(currency='rub', units=0, nano=0), total_amount_currencies=MoneyValue(currency='rub', units=24845, nano=0), total_amount_futures=MoneyValue(currency='rub', units=0, nano=0), expected_yield=Quotation(units=-2, nano=590000000), positions=[PortfolioPosition(figi='BBG0047315D0', instrument_type='share', quantity=Quotation(units=600, nano=0), average_position_price=MoneyValue(currency='rub', units=33, nano=0), expected_yield=Quotation(units=2082, nano=0), current_nkd=MoneyValue(currency='', units=0, nano=0), average_position_price_pt=Quotation(units=0, nano=0), current_price=MoneyValue(currency='rub', units=36, nano=470000000), average_position_price_fifo=MoneyValue(currency='rub', units=33, nano=0), quantity_lots=Quotation(units=6, nano=0), blocked=False, blocked_lots=Quotation(units=0, nano=0), position_ui

In [20]:
actives = []
for pos in r.positions:
    actives.append(portfolio_pose_todict(pos))


In [21]:
actives[0], len(actives)

({'figi': 'BBG0047315D0',
  'quantity': 600.0,
  'expected_yield': 2082.0,
  'instrument_type': 'share',
  'average_buy_price': 33.0,
  'currency': 'rub',
  'nkd': 0.0,
  'sell_sum': 21882.0,
  'comission': 65.646,
  'tax': 27.066},
 6)

In [22]:
cast_money(r.total_amount_portfolio)

89621.82

In [23]:
df[df['figi'] == 'TCS00A0ZZBC2']

,Unnamed: 0,name,figi,ticker,class_code,isin,currency,lot,cap
116,213,Софтлайн,TCS00A0ZZBC2,SOFL,TQBR,RU000A0ZZBC2,rub,10,39.99


## Покупка/продажа

In [26]:
from tinkoff.invest import MoneyValue
from tinkoff.invest import OrderDirection, OrderType

In [70]:


with Client(token) as client:
    resp = client.orders.post_order(figi = 'BBG004730N88', quantity = 1, price = MoneyValue(units=281, nano=0), 
    direction = OrderDirection.ORDER_DIRECTION_BUY, account_id = "todo", order_type = OrderType.ORDER_TYPE_LIMIT)
resp

PostOrderResponse(order_id='45804904891', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_NEW: 4>, lots_requested=1, lots_executed=0, initial_order_price=MoneyValue(currency='rub', units=2810, nano=0), executed_order_price=MoneyValue(currency='rub', units=0, nano=0), total_order_amount=MoneyValue(currency='rub', units=0, nano=0), initial_commission=MoneyValue(currency='rub', units=1, nano=410000000), executed_commission=MoneyValue(currency='rub', units=0, nano=0), aci_value=MoneyValue(currency='', units=0, nano=0), figi='BBG004730N88', direction=<OrderDirection.ORDER_DIRECTION_BUY: 1>, initial_security_price=MoneyValue(currency='rub', units=281, nano=0), order_type=<OrderType.ORDER_TYPE_LIMIT: 1>, message='', initial_order_price_pt=Quotation(units=0, nano=0), instrument_uid='e6123145-9665-43e0-8413-cd61b8aa9b13', order_request_id='', response_metadata=ResponseMetadata(tracking_id='2c236ee69d0bb6d87bd5f416c0b84aa1', server_time=datetime.datetime(2024, 4, 15, 

In [36]:
resp.execution_report_status == 3

True

In [7]:
df[df['ticker'] == 'YNDX']['lot'].item()

1

In [14]:
def get_acc_id(token):
    with Client(token) as client:
        r = client.users.get_accounts()
    ids = []
    for elem in r.accounts:
        ids.append(elem.id)
    return ids

In [10]:
df[df['ticker'] == 'AFKS']['figi'].values[0]

'BBG004S68614'

In [17]:
a = 0.000001
print(int(a), a - int(a))

0 1e-06


In [18]:
black_list = ['IRAO', 'HYDR', 'VTBR', 'UNAC']

In [19]:
def trade_stock(acc_id, token, quantity, price, direction, ticker = None, figi = None,):
    if ticker is None and figi is None:
        raise KeyError('Ticker or figi must be provided')
    # Определяем тип сделки: покупка или продажа
    if direction == 'buy':  
        direct = OrderDirection.ORDER_DIRECTION_BUY
    elif direction == 'sell':
        direct = OrderDirection.ORDER_DIRECTION_SELL
    else:
        raise KeyError('Direction must be "buy" or "sell"')
    # Получаем figi по тикеру
    if figi is None:
        try:
            figi = df[df['ticker'] == ticker]['figi'].values[0]
        except:
            raise KeyError('Ticker not in Companies DF')
    else:
        figi = figi
    # Получаем значение цены в терминах Tinkoff API
    deal_price = MoneyValue(units = int(price), nano = int(1e9 * (price - int(price))))
    
    with Client(token) as client:
        resp = client.orders.post_order(figi = figi, quantity = quantity, price = deal_price, 
            direction = direct, account_id = acc_id, order_type = OrderType.ORDER_TYPE_LIMIT)
    return resp

In [21]:
import datetime

week_day = datetime.datetime.now().weekday()

if week_day in [5, 6]:
    print('Today is a weekend')
    #todo SKIP
week_day

1

In [40]:

portfolio, opers = run()    # Получение портфолио и операций

balance = cast_money(portfolio.total_amount_portfolio)

actives = []                # список активов
currencies = []
for pos in portfolio.positions:
    active = portfolio_pose_todict(pos)
    if active['instrument_type'] == 'share':
        actives.append(active)
    if active['instrument_type'] == 'currency':
        currencies.append(active) 

len(actives), balance

(4, 89510.31)

In [41]:
currencies

[{'figi': 'RUB000UTSTOM',
  'quantity': 24839.07,
  'expected_yield': 0.0,
  'instrument_type': 'currency',
  'average_buy_price': 1.0,
  'currency': 'rub',
  'nkd': 0.0,
  'sell_sum': 24839.07,
  'comission': 74.51721,
  'tax': 0},
 {'figi': 'BBG0013HJJ31',
  'quantity': 0.06,
  'expected_yield': 2.18,
  'instrument_type': 'currency',
  'average_buy_price': 62.8875,
  'currency': 'rub',
  'nkd': 0.0,
  'sell_sum': 5.953250000000001,
  'comission': 0.01785975,
  'tax': 0.02834}]

In [ ]:
df[df['figi'] == resp.figi]['lot'].item()

In [71]:
import pytz
def get_prices_for_actives(actives):
    prices = dict()
    figs = [stock['figi'] for stock in actives]
    with Client(token) as client:
            #for stock in actives
            u = client.market_data.get_last_prices(figi=figs)
    prices['time'] = datetime.datetime.now(pytz.utc)
    for resp in u.last_prices:
        price = cast_money(resp.price)
        lot = df[df['figi'] == resp.figi]['lot'].item()
        prices[resp.figi] = {'price' : price, 'lot' : lot, 'total' : price * lot}
    return prices

In [44]:
actives

[{'figi': 'BBG0047315D0',
  'quantity': 600.0,
  'expected_yield': 2142.0,
  'instrument_type': 'share',
  'average_buy_price': 33.0,
  'currency': 'rub',
  'nkd': 0.0,
  'sell_sum': 21942.0,
  'comission': 65.82600000000001,
  'tax': 27.846},
 {'figi': 'BBG003LYCMB1',
  'quantity': 12.0,
  'expected_yield': -2112.0,
  'instrument_type': 'share',
  'average_buy_price': 1675.0,
  'currency': 'rub',
  'nkd': 0.0,
  'sell_sum': 17988.0,
  'comission': 53.964,
  'tax': 0},
 {'figi': 'TCS00A0ZZBC2',
  'quantity': 7.0,
  'expected_yield': 1280.93,
  'instrument_type': 'share',
  'average_buy_price': 0.02,
  'currency': 'rub',
  'nkd': 0.0,
  'sell_sum': 1281.0700000000002,
  'comission': 3.8432100000000005,
  'tax': 16.65209},
 {'figi': 'TCS00A107J11',
  'quantity': 74.0,
  'expected_yield': -2708.4,
  'instrument_type': 'share',
  'average_buy_price': 353.55,
  'currency': 'rub',
  'nkd': 0.0,
  'sell_sum': 23454.3,
  'comission': 70.3629,
  'tax': 0}]

In [45]:
def check_stocks_to_sell(actives, acc_id, token, predictions):
    prices = get_prices_for_actives(actives)
    order_responses = []
    for stock in actives:
        change = 100 * abs(prices[stock['figi']] - stock['average_buy_price']) /  stock['average_buy_price']
        try:
            ticker = df[df['figi'] == stock['figi']]['ticker'].values[0]
        except:
            raise KeyError("Can't get ticker via figi")
        if change > 3 or predictions[ticker] <= -80:
            resp = trade_stock(acc_id, token, stock['quantity'], prices[stock['figi']], 'sell', figi = stock['figi'])
            order_responses.append(resp)
    return order_responses

In [95]:
def preds_to_orders(predictions, acc_id, token, currencies): # predictions - a dict like "figi" :  prediction
    to_buy = []
    total = 0
    for pair in predictions:
        if predictions[pair] >= 80:
            total += predictions[pair]
            to_buy.append({'figi' : pair , 'prediction' : predictions[pair]})
    to_buy.sort(key = lambda x: x['prediction'], reverse = True)
    for i, stock in enumerate(to_buy):
        to_buy[i]['weight'] = stock['prediction'] / total

    # CHECKPOINT 1
    # return to_buy
    free_balance = 0
    for curr in currencies:
        if curr['figi'] == 'RUB000UTSTOM':
            free_balance = curr['quantity']
    if free_balance <= 0:
        print('Balance equals to zero or less')
        return None

    cur_prices = get_prices_for_actives(to_buy)
    rest_balance = free_balance
    # CHECKPOINT 2
    # return cur_prices
    
    for i, stock in enumerate(to_buy):
        total = cur_prices[stock['figi']]['total']
        fraction = free_balance * stock['weight']
        quantity = int(fraction  / total)
        if quantity * total <= rest_balance:
            to_buy[i]['quantity'] = quantity
            rest_balance -= quantity * total
        else:
            to_buy[i]['quantity'] = 0
    
    # CHECKPOINT 3
    # return to_buy
    
    buy_resps = []
    for stock in to_buy:
        resp = trade_stock(acc_id, token, stock['quantity'], cur_prices[stock['figi']]['price'] - 10, 'buy', figi = stock['figi'])
        buy_resps.append(resp)
    return buy_resps
        
    

In [96]:
predictions = {'BBG004730N88' : 100, 'BBG00QPYJ5H0' : 60, 'BBG004731032' : 140}
preds_to_orders(predictions, get_acc_id(token)[0], token, currencies)

[PostOrderResponse(order_id='46740515562', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_NEW: 4>, lots_requested=1, lots_executed=0, initial_order_price=MoneyValue(currency='rub', units=7698, nano=0), executed_order_price=MoneyValue(currency='rub', units=0, nano=0), total_order_amount=MoneyValue(currency='rub', units=0, nano=0), initial_commission=MoneyValue(currency='rub', units=293, nano=850000000), executed_commission=MoneyValue(currency='rub', units=0, nano=0), aci_value=MoneyValue(currency='', units=0, nano=0), figi='BBG004731032', direction=<OrderDirection.ORDER_DIRECTION_BUY: 1>, initial_security_price=MoneyValue(currency='rub', units=7698, nano=0), order_type=<OrderType.ORDER_TYPE_LIMIT: 1>, message='', initial_order_price_pt=Quotation(units=0, nano=0), instrument_uid='02cfdf61-6298-4c0f-a9ca-9cabc82afaf3', order_request_id='', response_metadata=ResponseMetadata(tracking_id='b4a7ccd45f5b4f04b36de283daffe271', server_time=datetime.datetime(2024, 5, 

In [38]:
white_list = {'YNDX': 0.2,
              'TCSG': 0.5,
              'SBER': 0.01,
              'LKOH': 0.5,
              'ROSN': 0.05,
              'BSPB': 0.01,
              'MAGN': 0.005,
              'MGNT': 0.5,
              'PHOR': 1,
              'ALRS': 0.01,
              'PLZL': 0.5,
              'SNGSP': 0.005,
              'MTSS': 0.05,
              'OZON': 0.5,
              'BANE': 0.5,
              'FIVE': 0.5,
              'SNGS': 0.005,
              'TATN': 0.1,
              'PIKK': 0.1,
              'NVTK': 0.2,
              'CHMF': 0.2,
              'SMLT': 0.5,
              'UPRO': 0.001,
              'SVCB': 0.005,
              'LENT': 0.5,
              'AFKS': 0.001,
              'ASTR': 0.05,
              'RTKM': 0.01,
              'HHRU': 1,
              'POSI': 0.2}


In [40]:
for i, elem in enumerate(white_list):
    print(i, elem)

0 YNDX
1 TCSG
2 SBER
3 LKOH
4 ROSN
5 BSPB
6 MAGN
7 MGNT
8 PHOR
9 ALRS
10 PLZL
11 SNGSP
12 MTSS
13 OZON
14 BANE
15 FIVE
16 SNGS
17 TATN
18 PIKK
19 NVTK
20 CHMF
21 SMLT
22 UPRO
23 SVCB
24 LENT
25 AFKS
26 ASTR
27 RTKM
28 HHRU
29 POSI
